In [10]:
import sqlalchemy
sqlalchemy.create_engine('sqlite:///factbook.db')
%load_ext sql
%sql sqlite:///factbook.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: @factbook.db'

# Data Overview

We'll take a quick look at the first 5 rows of the database to get an idea of the data contained in it.

In [11]:
%%sql
SELECT *
FROM facts
LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Here are the descriptions for some of the columns:

1. name - The name of the country.
2. area - The total land and sea area of the country.
3. population - The country's population.
4. population_growth- The country's population growth as a percentage.
5. birth_rate - The country's birth rate, or the number of births a year per 1,000 people.
6. death_rate - The country's death rate, or the number of death a year per 1,000 people.
7. area- The country's total area (both land and water).
8. area_land - The country's land area in square kilometers.
9. area_water - The country's waterarea in square kilometers.

# Summary Stats

In [12]:
%%sql
SELECT 
    MIN(population), 
    MAX(population), 
    MIN(population_growth), 
    MAX(population_growth)
FROM facts

 * sqlite:///factbook.db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


There is a country with a population of 0 and a country with a population of 7256490011.
I'll identify those 2 countries below

In [17]:
%%sql
SELECT *
FROM facts
WHERE population == (SELECT MIN(population) from facts);
    

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


In [18]:
%%sql
SELECT *
FROM facts
WHERE population == (SELECT MAX(population) from facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


There is a row for the entire world population combined. In future analysis we wshould exclude this row

In [20]:
%%sql
SELECT 
    MIN(population),
    MAX(population),
    MIN(population_growth),
    MAX(population_growth)
FROM facts
WHERE name <> 'World';

 * sqlite:///factbook.db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,1367485388,0.0,4.02


In [24]:
%%sql
SELECT *
FROM facts
WHERE population == (SELECT MAX(population) FROM facts WHERE name <> 'World');

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44


The country with the largest population is China, with a 1.3 billion population

# Average Population and Area

Below I will explore density, which depends upon a country's area and population.

In [29]:
%%sql
SELECT AVG(population) as avg_pop, AVG(area) AS avg_area
FROM facts
WHERE name <> 'World';

 * sqlite:///factbook.db
Done.


avg_pop,avg_area
32242666.56846473,555093.546184739


The average population is 32 million and the average area of each country is 555,000 square kilometers.

In the below query I will search for countries whose population is above the world average as well as having a total area that is less than the world average. Density is calculated according to population and land mass.

In [33]:
%%sql
SELECT *
FROM facts
WHERE population > (SELECT AVG(population) from facts WHERE name <> 'World')
AND area < (SELECt AVG(area) FROM facts WHERE name <> 'World')


 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31
